In [365]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA

In [366]:
# Load your credit card fraud detection dataset
data = pd.read_csv('creditcard.csv')
data


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [367]:
#data1 is class=1 data
data1 = data[data['Class']==1]
data1

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [375]:
#take 800 class=0 data and join it with all class =1 data and shuffle the data
data0 = data[data['Class'] == 0].sample(800)
data0
#data0+data1
data = pd.concat([data0, data1])
data        
#shuffle randomly 'data dataframe'
data = data.sample(frac=1).reset_index(drop=True)
data


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,65728.0,1.227614,-0.668974,-0.271785,-0.589440,-0.604795,-0.350285,-0.486365,-0.010809,-0.794944,...,-0.026055,-0.295255,-0.180459,-0.436539,0.494649,-0.283738,-0.001128,0.035075,98.01,1
1,163874.0,0.087143,0.835113,-0.004650,-0.744765,0.719321,-0.719058,1.017592,-0.163294,0.006402,...,-0.303555,-0.701959,-0.003140,-0.663380,-0.396303,0.181098,0.244251,0.090629,8.73,0
2,83734.0,-10.486436,8.879636,-6.599708,-0.173357,-4.262577,-0.356341,-4.985335,-0.598359,2.330486,...,1.100265,-0.186045,1.454489,0.058808,0.474443,-0.363645,0.879679,0.991215,9.49,0
3,27891.0,1.175698,-0.013785,0.554003,0.587513,-0.752971,-0.838451,-0.184675,0.063575,0.139209,...,-0.212359,-0.801090,0.166947,0.464232,0.072677,0.071234,-0.051050,0.007972,14.99,0
4,96717.0,-3.705856,4.107873,-3.803656,1.710314,-3.582466,1.469729,-9.621560,-11.913105,-0.322297,...,-5.498772,2.941475,0.916236,-0.255504,-0.183835,-0.584539,-0.315484,-0.097223,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,162181.0,-0.513915,-0.573346,2.115486,-3.135314,-0.801017,0.337765,-0.615006,0.136381,-1.680142,...,-0.198032,-0.150554,-0.390303,-1.020012,0.497382,-0.146285,0.072764,0.057278,24.90,0
1288,120313.0,-1.611515,0.120133,1.562873,0.298314,-0.235515,-0.431513,-0.287207,0.498691,0.204004,...,0.299546,0.809447,-0.250912,0.024013,0.643027,0.134738,-0.136975,-0.022183,32.89,0
1289,64656.0,-4.739801,3.916937,-0.395545,-2.935347,0.056756,0.147893,0.788090,-0.087901,4.118853,...,-0.819225,-0.580392,-0.059203,-1.214707,0.642477,0.809135,1.853068,0.713184,1.54,0
1290,146022.0,0.908637,2.849024,-5.647343,6.009415,0.216656,-2.397014,-1.819308,0.338527,-2.819883,...,0.407260,-0.397435,-0.080006,-0.168597,0.465058,0.210510,0.648705,0.360224,1.18,1


In [376]:
# Extract features (v1 to v28) and labels (class)
features = data.iloc[:, 1:-1].values
labels = data.iloc[:, -1].values


In [377]:
# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [378]:
# Apply PCA to reduce dimensionality
pca = PCA(n_components=2)  # Choose the number of components you want to keep
features_pca = pca.fit_transform(features_scaled)

In [379]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_pca, labels, test_size=0.2, random_state=42)

In [380]:
n_qubits = 2

In [395]:
# Define the quantum device
dev = qml.device("lightning.qubit", wires=n_qubits)

In [396]:
# Quantum circuit
@qml.qnode(dev)
def circuit(params, x):
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

In [397]:
# Number of layers in the quantum circuit
num_layers = 3  # You can adjust this as needed

In [398]:
# Cost function for SVM
def cost(params, x, y):
    predictions = [circuit(params, x[i]) for i in range(len(x))]
    return np.sum((predictions - y) ** 2)

In [399]:
# Initialize parameters
params = np.random.rand(num_layers, n_qubits, 3)
params.shape

(3, 2, 3)

In [400]:
# Optimize the circuit parameters
opt = qml.GradientDescentOptimizer(0.1)
for step in range(10):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)

In [401]:
# Make predictions on the test set
predictions = [np.sign(circuit(params, x)) for x in X_test]


In [402]:
# Evaluate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Classification Accuracy:", accuracy)
#evaluate f1 score
f1 = f1_score(y_test, predictions)
print("F1 Score:", f1)

Classification Accuracy: 0.38223938223938225
F1 Score: 0.553072625698324
